In [2]:
using JuMP, Cbc, NamedArrays

# Sets
T = [1, 2, 3, 4, 5, 6, 7]
S = [1, 2, 3, 4, 5]

# Parameters
w = Dict(zip(T, [10, 8, 12, 10, 6, 12, 10]))
b = Dict(zip(T, [6, 10, 6, 9, 9, 7, 10]))
a = NamedArray([1 1 1 0 0 0 0; 0 1 1 1 0 0 0; 0 0 1 1 1 0 0; 0 0 0 1 1 1 0; 0 0 0 0 1 1 1], (S,T), ("shift","time"))

# General model
m = Model()

@variable(m, x[S] >= 0)
@variable(m, y[S] >= 0)
@variable(m, z[T] >= 0)

@objective(m, Min, sum(42*x[i] + 60*y[i] for i in S) + sum(16*z[j] for j in T))

@constraint(m, waiters[j in T], sum(a[i,j]x[i] for i in S) - z[j] >= w[j])
@constraint(m, bussers[j in T], sum(a[i,j]y[i] for i in S) + z[j] >= b[j])

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

Presolve 14 (0) rows, 17 (0) columns and 44 (0) elements
Perturbing problem by 0.001% of 60 - largest nonzero change 0.00035269088 ( 0.00083974018%) - largest zero change 0
0  Obj 0 Primal inf 125 (14)
19  Obj 2748.0065
Optimal - objective value 2748
Optimal objective 2748 - 19 iterations time 0.022


* Solver : COIN Branch-and-Cut (Cbc)

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Result count       : 1
  Has duals          : false
  Message from the solver:
  "Cbc_status          = finished - check isProvenOptimal or isProvenInfeasible to see if solution found (or check value of best solution)
Cbc_secondaryStatus = unset (status_ will also be -1)
"

* Candidate solution
  Objective value      : 2748.0
  Objective bound      : 2748.0
  Primal solution :
    x[1] : 10.0
    x[2] : 0.0
    x[3] : 14.0
    x[4] : 0.0
    x[5] : 12.0
    y[1] : 6.0
    y[2] : 4.0
    y[3] : 1.0
    y[4] : 0.0
    y[5] : 8.0
    z[1] : 0.0
    z[2] : 0.0
    z[3] : 0.0
    z[4] : 4.0
    z[5] : 0.0
    z[6] : 0.0
    z[7] : 2.0

* Work counters
  Solve time (sec)   : 0.02400


In [4]:
# Just for fun, we will see how different the cost is if we don't allow waiters to bus tables.
# We can effectively achieve this here by getting rid of our z variable, and leaving everything else the same

using JuMP, Cbc, NamedArrays

# Sets
T = [1, 2, 3, 4, 5, 6, 7]
S = [1, 2, 3, 4, 5]

# Parameters
w = Dict(zip(T, [10, 8, 12, 10, 6, 12, 10]))
b = Dict(zip(T, [6, 10, 6, 9, 9, 7, 10]))
a = NamedArray([1 1 1 0 0 0 0; 0 1 1 1 0 0 0; 0 0 1 1 1 0 0; 0 0 0 1 1 1 0; 0 0 0 0 1 1 1], (S,T), ("shift","time"))

# General model
m = Model()

@variable(m, x[S] >= 0)
@variable(m, y[S] >= 0)
#@variable(m, z[T] >= 0)

@objective(m, Min, sum(42*x[i] + 60*y[i] for i in S))

#@constraint(m, waiters[j in T], sum(a[i,j]x[i] for i in S) - z[j] >= w[j])
#@constraint(m, bussers[j in T], sum(a[i,j]y[i] for i in S) + z[j] >= b[j])
@constraint(m, waiters[j in T], sum(a[i,j]x[i] for i in S) >= w[j])
@constraint(m, bussers[j in T], sum(a[i,j]y[i] for i in S) >= b[j])

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

Presolve 5 (-9) rows, 7 (-3) columns and 10 (-20) elements
0  Obj 1800 Primal inf 26.999995 (5)
3  Obj 2760
Optimal - objective value 2760
After Postsolve, objective 2760, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2760 - 3 iterations time 0.012, Presolve 0.01


* Solver : COIN Branch-and-Cut (Cbc)

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Result count       : 1
  Has duals          : false
  Message from the solver:
  "Cbc_status          = finished - check isProvenOptimal or isProvenInfeasible to see if solution found (or check value of best solution)
Cbc_secondaryStatus = unset (status_ will also be -1)
"

* Candidate solution
  Objective value      : 2760.0
  Objective bound      : 2760.0
  Primal solution :
    x[1] : 10.0
    x[2] : 8.0
    x[3] : 0.0
    x[4] : 2.0
    x[5] : 10.0
    y[1] : 6.0
    y[2] : 9.0
    y[3] : 0.0
    y[4] : 0.0
    y[5] : 10.0

* Work counters
  Solve time (sec)   : 0.00500


In [ ]:
#looks like we save $12 if we let waiters bus tables when needed, adjusting their pay as necessary